Importing Libraries needed to run script

In [1]:
import os
import numpy as np
import pandas as pd1
import shutil
import csv

In [2]:
models = os.listdir("Modules") #modules list as array 
print(models)

['adder_1bit.v', 'Barrel_shifter.v', 'binary_multiplier.v', 'FSM_mealy.v', 'FSM_moore.v', 'logic_gate.v', 'ripple_adder.v', 'shifter.v', 'universal_register.v']


In [3]:
os.system("vivado -mode batch -source tcl_create.tcl")   #creating new project

0

In [4]:
for mod in models:
    os.system("vivado -mode batch -source  tcl_add.tcl -tclargs {}".format(mod))

In [5]:
os.mkdir("results")

In [6]:
for filename in models:
    os.mkdir("results/"+filename[:-2])
    os.system("vivado -mode batch -source  tcl_run.tcl -tclargs {}".format(filename))
    print("{} reported".format(filename)) 

adder_1bit.v reported
Barrel_shifter.v reported
binary_multiplier.v reported
FSM_mealy.v reported
FSM_moore.v reported
logic_gate.v reported
ripple_adder.v reported
shifter.v reported
universal_register.v reported


Generating Result.csv file 
The approach is to search for certain text in the generated reports and extract out the numerical values needed.

In [7]:
data_files = os.listdir("results")  
print(data_files)

['adder_1bit', 'Barrel_shifter', 'binary_multiplier', 'FSM_mealy', 'FSM_moore', 'logic_gate', 'ripple_adder', 'shifter', 'universal_register']


In [8]:
file_types = ['power.txt','timing.txt','utilization.txt']

req_data = ['Total On-Chip Power','Data Path Delay','Slice LUTs']

Power = []
Delay = []
LUTs = []

In [9]:

for data in data_files :
    for file in file_types:
        with open("results/{}/{}".format(data,file),'r') as File:
             content = File.read()

        words = content.split('\n')   

        for req_word in words:
            if file_types[0] in file:
                if req_data[0] in req_word:
                    pwr = float(req_word.split()[6])
                    print(pwr)
                    Power.append(pwr)
  

        for req_word in words:
            if file_types[1] in file:
                if req_data[1] in req_word:
                    delay = float(req_word.split()[3][:-2])
                    print(delay)
                    Delay.append(delay)


        for req_word in words:
            if file_types[2] in file:
                if req_data[2] in req_word:
                    luts = float(req_word.split()[4])
                    print(luts) 
                    LUTs.append(luts) 
                    print("_____")          

             




1.034
5.896
1.0
_____
1.389
6.233
4.0
_____
13.717
11.621
68.0
_____
0.338
4.356
5.0
_____
0.394
4.204
5.0
_____
1.389
6.233
4.0
_____
1.389
6.233
4.0
_____
1.408
4.419
3.0
_____
1.898
4.304
9.0
_____


In [10]:
Final_results = [Power,Delay,LUTs]
row_name = ['Power','Delay','LUTs']

df = pd1.DataFrame(Final_results, index = row_name)
df.columns = data_files

df.to_csv('Final_results.csv', index = row_name)
print(df)

       adder_1bit  Barrel_shifter  binary_multiplier  FSM_mealy  FSM_moore  \
Power       1.034           1.389             13.717      0.338      0.394   
Delay       5.896           6.233             11.621      4.356      4.204   
LUTs        1.000           4.000             68.000      5.000      5.000   

       logic_gate  ripple_adder  shifter  universal_register  
Power       1.389         1.389    1.408               1.898  
Delay       6.233         6.233    4.419               4.304  
LUTs        4.000         4.000    3.000               9.000  
